In [1]:
from ast import increment_lineno
import numpy as np
import pandas as pd
import seaborn as sns 
import tensorflow as tf 

from tensorflow import keras as ks
from tensorflow.keras.layers import Dense
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

%matplotlib inline 

In [ ]:
%pip install keras-tuner

In [3]:
import keras_tuner as kt

In [ ]:
boston_ds = datasets.load_boston()
columns = boston_ds.feature_names
target = boston_ds.target
data = pd.DataFrame(boston_ds.data, columns=columns).astype(np.float32)
data.head()

In [5]:
data['Price'] = target.astype(np.float32)
target = data['Price']
# data.head()

In [ ]:
data.isnull().sum()

In [ ]:
plt.figure(figsize=(11.7,8.27))
sns.displot(data['Price'], kde=True, height=7, aspect=2, bins=30)
plt.show()

In [ ]:
corr = data.corr().round(2)
plt.figure(figsize=(12.7,8.27))
sns.heatmap(data=corr, annot=True)

In [ ]:
features = ['RM', 'LSTAT']
plt.figure(figsize=(15.7,5.27))
for i, col in enumerate(features):
    plt.subplot(1, len(features) , i+1)
    x = data[col]
    y = target
    plt.scatter(x, y, marker='o')
    plt.title(col)
    plt.xlabel(col)
    plt.ylabel('Price')

In [6]:
X = pd.DataFrame(np.c_[data['LSTAT'], data['RM']], columns = ['LSTAT','RM'])
Y = data['Price']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=5)

In [8]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
mms.fit(X_train)
X_train = mms.transform(X_train)
X_test = mms.transform(X_test)

In [9]:
from tensorflow.keras.models import Sequential

def model_builder(hp):
  model = Sequential(name='BostonHousingPrices')
  # automate unites hp
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(Dense(units=hp_units, input_dim = 2, activation='relu', name='input'))
  for i in range(1, hp.Int("num_layers", 2, 6)):
    model.add(
        tf.keras.layers.Dense(
            units=hp.Int("units_" + str(i), min_value=32, max_value=512, step=32),
            activation="relu", name='Hidden-' + str(i)))
    
  model.add(Dense(1, activation='linear', name='Output'))
  # automate learning_rate hp
  lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                loss='mean_absolute_error', metrics=['mae'])
  return model

In [ ]:
tuner = kt.RandomSearch(
    hypermodel=model_builder,
    objective="val_loss",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="/content/hp_search_results/",
    project_name="TF_SLR",
)
tuner.search_space_summary()

In [ ]:
tuner.search(X_train, y_train, epochs=50, validation_split=0.2)

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete.\n
Optimal No of Layers: {best_hps.get('num_layers')}\n
Optimal Learning-Rate: {best_hps.get('learning_rate')}
""")

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)
from keras import callbacks
es = callbacks.EarlyStopping(monitor ="val_loss",
                             mode="min", patience=5,
                             restore_best_weights=True)
hist = hypermodel.fit(X_train, y_train, epochs=30, validation_split=0.2, callbacks=[es])

In [14]:
hypermodel.save("slr_model.h5")

In [15]:
import tempfile
import os

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    hypermodel,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)
print('\nSaved model:')
!ls -l {export_path}

export_path = /tmp/1


Saved model:
total 108
drwxr-xr-x 2 root root  4096 Oct 25 08:58 assets
-rw-r--r-- 1 root root  8328 Oct 25 08:58 keras_metadata.pb
-rw-r--r-- 1 root root 89739 Oct 25 08:58 saved_model.pb
drwxr-xr-x 2 root root  4096 Oct 25 08:58 variables


In [19]:
figure = plt.gcf()
figure.set_size_inches((15, 5))
plt.title('Loss - Epoch Graphs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(range(1, len(hist.history['loss']) + 1), hist.history['loss'])
plt.plot(range(1, len(hist.history['val_loss']) + 1), hist.history['val_loss'])
plt.legend(['Loss', 'Validation Loss'])
plt.show()

In [20]:
figure = plt.gcf()
figure.set_size_inches((15, 5))
plt.title('Mean Absolute Error - Epoch Graph')
plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error')
plt.plot(range(1, len(hist.history['loss']) + 1), hist.history['loss'])
plt.plot(range(1, len(hist.history['val_loss']) + 1), hist.history['val_loss'])
plt.legend(['Mean Absolute Error', 'Validation Mean Absolute Error'])
plt.show()

In [ ]:
result = hypermodel.evaluate(X_test, y_test)
print(result)
for i in range(len(result)):
    print(f'{hypermodel.metrics_names[i]} ---> {result[i]}')

In [22]:
prediction_data = np.array([6.0910, 10.46])
prediction_data = mms.transform(prediction_data.reshape(1, -1))
predict_result = model.predict(prediction_data)
print(f'Predicted price: {predict_result[0, 0]}')